# Chapter 4.2

## Importing Libraries

In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import copy
import tqdm
from hfunc import models
from hfunc import metrics
import time

## Self-created functions

In [3]:
def node_pruning(model, tester_model, x, y, layer_sizes, tol, ignore_cutoff, method='exhaustive'):

    loss, acc = model.evaluate(x, y, verbose=0, batch_size=256)
    original = model.get_weights()
    weight_len = len(original) - 3
    bas = [acc]
    bls = [loss]
    best_weights = model.get_weights()
    best_acc = 0
    best_loss = 1e20
    ol = loss
    oa = acc
    amounts = []
    places = []

    for layer, size in enumerate(layer_sizes):
        end_not_reached = True
        num_removed = 0
        nodes_removed = []
        if method == 'exhaustive':
            current_pos = 0
            best_change = tol
            best_pos = -1
            improved = False
            while end_not_reached or improved:
                if not(end_not_reached):
                    end_not_reached = True
                    improved = False
                    current_pos = 0
                    size -= 1
                    nodes_removed += [best_pos]
                    best_weights[weight_len - (2*layer+1)][...,best_pos] = 0
                    best_weights[weight_len - 2*layer][best_pos] = 0
                    best_pos = -1
                    ol = best_loss
                    oa = best_acc
                    bas += [best_acc]
                    bls += [best_loss]
                    best_change = tol
                    num_removed += 1
                if current_pos in nodes_removed:
                    current_pos += 1
                    if current_pos - num_removed >= size:
                        end_not_reached = False
                    continue
                w = copy.deepcopy(best_weights)
                w[weight_len - (2*layer+1)][...,current_pos] = 0
                w[weight_len - 2*layer][current_pos] = 0
                tester_model.set_weights(w)
                nl, na = tester_model.evaluate(x, y, verbose=0, batch_size=256)
                if ol - nl >= best_change:
                    best_change = ol - nl
                    best_pos = current_pos
                    improved = True
                    best_acc = na
                    best_loss = nl
                current_pos += 1
                if current_pos - num_removed >= size:
                    end_not_reached = False
        elif method == 'greedy':
            nodes_to_estimate = list(np.arange(size))
            current_pos = nodes_to_estimate[0]
            idx = 0
            while end_not_reached:
                w = copy.deepcopy(best_weights)
                w[weight_len - (2*layer+1)][...,current_pos] = 0
                w[weight_len - 2*layer][current_pos] = 0
                tester_model.set_weights(w)
                nl, na = tester_model.evaluate(x, y, verbose=0, batch_size=256)
                if ol - nl >= tol:
                    oa = na
                    ol = nl
                    size -= 1
                    nodes_removed += [current_pos]
                    nodes_to_estimate.remove(current_pos)
                    best_weights[weight_len - (2*layer+1)][..., current_pos] = 0
                    best_weights[weight_len - 2*layer][current_pos] = 0
                    bas += [oa]
                    bls += [ol]
                    num_removed += 1
                    idx = 0
                elif ol - nl <= ignore_cutoff:
                    size -= 1
                    nodes_to_estimate.remove(current_pos)
                else:
                    idx += 1
                if idx >= size:
                    end_not_reached = False
                else:
                    current_pos = nodes_to_estimate[idx]
        amounts.append(num_removed)
        places.append(nodes_removed)

    return best_weights, bas, bls, amounts, places

## Multi-layer Perceptron

### MNIST

In [4]:
np.random.seed(2020)

In [5]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [6]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [ ]:
trials = 10
tol = -1e-5
layer_sizes = [32, 64, 128]
num_rem_ex = []
num_rem_greed = []
num_rem_greed2 = []
delta_accs_ex = []
delta_losses_ex = []
delta_accs_greed = []
delta_losses_greed = []
delta_accs_greed2 = []
delta_losses_greed2 = []
diff_accs = []
diff_losses = []
diff_rem = []
diff_rem2 = []
diff_accs2 = []
diff_losses2 = []
time_ex = []
time_greed = []
time_greed2 = []

for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    l, a = model.evaluate(x_test, y_test, verbose=0)
    
    ig_cut = None
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut)
    end = time.time()
    num_rem_ex += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_ex += [na - a]
    delta_losses_ex += [nl - l]
    time_ex += [end - start]
    
    ig_cut = -1e-2
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
    end = time.time()
    num_rem_greed += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_greed += [na - a]
    delta_losses_greed += [nl - l]
    time_greed += [end - start]
    
    ig_cut = -1e-3
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
    end = time.time()
    num_rem_greed2 += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_greed2 += [na - a]
    delta_losses_greed2 += [nl - l]
    time_greed2 += [end - start]
    
    diff_accs += [delta_accs_ex[trial] - delta_accs_greed[trial]]
    diff_accs2 += [delta_accs_ex[trial] - delta_accs_greed2[trial]]
    diff_losses += [delta_losses_ex[trial] - delta_losses_greed[trial]]
    diff_losses2 += [delta_losses_ex[trial] - delta_losses_greed2[trial]]
    temp_rem = []
    temp_rem2 = []
    for i in range(len(layer_sizes)):   
        temp_rem += [num_rem_greed[trial][i] - num_rem_ex[trial][i]]
        temp_rem2 += [num_rem_greed2[trial][i] - num_rem_ex[trial][i]]
        
    diff_rem += [temp_rem]
    diff_rem2 += [temp_rem2]

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

In [ ]:
DR = pd.DataFrame(diff_rem, columns=["Layer 3", "Layer 2", "Layer 1"])
DR.to_csv('../../../results/num_removed_MLP_greed_ex_mnist.csv')
DR.describe()

In [ ]:
DR2 = pd.DataFrame(diff_rem2, columns=["Layer 3", "Layer 2", "Layer 1"])
DR2.to_csv('../../../results/num_removed_MLP_greed_ex_2_mnist.csv')
DR2.describe()

In [ ]:
A = pd.DataFrame([delta_accs_ex, delta_accs_greed, delta_accs_greed2])
A = A.T
A.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
A.to_csv('../../../results/change_accuracy_MLP_greed_ex_mnist.csv')
A.describe()

In [ ]:
L = pd.DataFrame([delta_losses_ex, delta_losses_greed, delta_losses_greed2])
L = L.T
L.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
L.to_csv('../../../results/change_loss_MLP_greed_ex_mnist.csv')
L.describe()

In [ ]:
DA = pd.Series(diff_accs)
DA.to_csv('../../../results/diff_accuracy_MLP_greed_ex_mnist.csv')
DA.describe()

In [ ]:
DA2 = pd.Series(diff_accs2)
DA2.to_csv('../../../results/diff_accuracy_MLP_greed_ex_2_mnist.csv')
DA2.describe()

In [ ]:
DL = pd.Series(diff_losses)
DL.to_csv('../../../results/diff_loss_MLP_greed_ex_mnist.csv')
DL.describe()

In [ ]:
DL2 = pd.Series(diff_losses2)
DL2.to_csv('../../../results/diff_loss_MLP_greed_ex_2_mnist.csv')
DL2.describe()

In [ ]:
Time = pd.DataFrame([time_ex, time_greed, time_greed2])
Time = Time.T
Time.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
Time.to_csv('../../../results/time_MLP_greed_ex_mnist.csv')
Time.describe()

In [ ]:
diff_rem = [sum(rem) for rem in diff_rem]

In [ ]:
diff_rem2 = [sum(rem) for rem in diff_rem2]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=diff_rem, y=diff_accs, marker='o', color='red', s=60)
plt.title("Accuracy difference in Greedy (-1e-2 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_accuracy_vs_nodes_removed_MLP_greed_ex_mnist.png")

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=diff_rem2, y=diff_accs2, marker='o', color='red', s=60)
plt.title("Accuracy difference in Greedy (-1e-3 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_accuracy_vs_nodes_removed_MLP_greed_ex_2_mnist.png")

In [ ]:
plt.scatter(x=diff_rem, y=diff_losses, marker='o', color='red', s=60)
plt.title("Loss difference in Greedy (-1e-2 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_loss_vs_nodes_removed_MLP_greed_ex_mnist.png")

In [ ]:
plt.scatter(x=diff_rem2, y=diff_losses2, marker='o', color='red', s=60)
plt.title("Loss difference in Greedy (-1e-3 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_loss_vs_nodes_removed_MLP_greed_ex_2_mnist.png")

In [ ]:
tol = -1e-5
ig_cut = -1e-3
layer_sizes = [32, 64, 128]
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
l, a = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
nl, na = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layers-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_MLP_greed_pru_mnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layers-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_MLP_greed_pru_mnist.png")

### Fashion MNIST

#### Set seed

In [ ]:
np.random.seed(2020)

In [ ]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [ ]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [ ]:
trials = 10
tol = -1e-5
layer_sizes = [32, 64, 128]
num_rem_ex = []
num_rem_greed = []
num_rem_greed2 = []
delta_accs_ex = []
delta_losses_ex = []
delta_accs_greed = []
delta_losses_greed = []
delta_accs_greed2 = []
delta_losses_greed2 = []
diff_accs = []
diff_losses = []
diff_rem = []
diff_rem2 = []
diff_accs2 = []
diff_losses2 = []
time_ex = []
time_greed = []
time_greed2 = []

for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    l, a = model.evaluate(x_test, y_test, verbose=0)
    
    ig_cut = None
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut)
    end = time.time()
    num_rem_ex += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_ex += [na - a]
    delta_losses_ex += [nl - l]
    time_ex += [end - start]
    
    ig_cut = -1e-2
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
    end = time.time()
    num_rem_greed += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_greed += [na - a]
    delta_losses_greed += [nl - l]
    time_greed += [end - start]
    
    ig_cut = -1e-3
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
    end = time.time()
    num_rem_greed2 += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_greed2 += [na - a]
    delta_losses_greed2 += [nl - l]
    time_greed2 += [end - start]
    
    diff_accs += [delta_accs_ex[trial] - delta_accs_greed[trial]]
    diff_accs2 += [delta_accs_ex[trial] - delta_accs_greed2[trial]]
    diff_losses += [delta_losses_ex[trial] - delta_losses_greed[trial]]
    diff_losses2 += [delta_losses_ex[trial] - delta_losses_greed2[trial]]
    temp_rem = []
    temp_rem2 = []
    for i in range(len(layer_sizes)):   
        temp_rem += [num_rem_greed[trial][i] - num_rem_ex[trial][i]]
        temp_rem2 += [num_rem_greed2[trial][i] - num_rem_ex[trial][i]]
        
    diff_rem += [temp_rem]
    diff_rem2 += [temp_rem2]

In [ ]:
DR = pd.DataFrame(diff_rem, columns=["Layer 3", "Layer 2", "Layer 1"])
DR.to_csv('../../../results/num_removed_MLP_greed_ex_fmnist.csv')
DR.describe()

In [ ]:
DR2 = pd.DataFrame(diff_rem2, columns=["Layer 3", "Layer 2", "Layer 1"])
DR2.to_csv('../../../results/num_removed_MLP_greed_ex_2_fmnist.csv')
DR2.describe()

In [ ]:
A = pd.DataFrame([delta_accs_ex, delta_accs_greed, delta_accs_greed2])
A = A.T
A.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
A.to_csv('../../../results/change_accuracy_MLP_greed_ex_fmnist.csv')
A.describe()

In [ ]:
L = pd.DataFrame([delta_losses_ex, delta_losses_greed, delta_losses_greed2])
L = L.T
L.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
L.to_csv('../../../results/change_loss_MLP_greed_ex_fmnist.csv')
L.describe()

In [ ]:
DA = pd.Series(diff_accs)
DA.to_csv('../../../results/diff_accuracy_MLP_greed_ex_fmnist.csv')
DA.describe()

In [ ]:
DA2 = pd.Series(diff_accs2)
DA2.to_csv('../../../results/diff_accuracy_MLP_greed_ex_2_fmnist.csv')
DA2.describe()

In [ ]:
DL = pd.Series(diff_losses)
DL.to_csv('../../../results/diff_loss_MLP_greed_ex_fmnist.csv')
DL.describe()

In [ ]:
DL2 = pd.Series(diff_losses2)
DL2.to_csv('../../../results/diff_loss_MLP_greed_ex_2_fmnist.csv')
DL2.describe()

In [ ]:
Time = pd.DataFrame([time_ex, time_greed, time_greed2])
Time = Time.T
Time.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
Time.to_csv('../../../results/time_MLP_greed_ex_fmnist.csv')
Time.describe()

In [ ]:
diff_rem = [sum(rem) for rem in diff_rem]

In [ ]:
diff_rem2 = [sum(rem) for rem in diff_rem2]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=diff_rem, y=diff_accs, marker='o', color='red', s=60)
plt.title("Accuracy difference in Greedy (-1e-2 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_accuracy_vs_nodes_removed_MLP_greed_ex_fmnist.png")

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=diff_rem2, y=diff_accs2, marker='o', color='red', s=60)
plt.title("Accuracy difference in Greedy (-1e-3 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_accuracy_vs_nodes_removed_MLP_greed_ex_2_fmnist.png")

In [ ]:
plt.scatter(x=diff_rem, y=diff_losses, marker='o', color='red', s=60)
plt.title("Loss difference in Greedy (-1e-2 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_loss_vs_nodes_removed_MLP_greed_ex_fmnist.png")

In [ ]:
plt.scatter(x=diff_rem2, y=diff_losses2, marker='o', color='red', s=60)
plt.title("Loss difference in Greedy (-1e-3 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_loss_vs_nodes_removed_MLP_greed_ex_2_fmnist.png")

In [ ]:
tol = -1e-5
ig_cut = -1e-3
layer_sizes = [32, 64, 128]
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
l, a = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
nl, na = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layers-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_MLP_greed_pru_fmnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layers-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_MLP_greed_pru_fmnist.png")

## Convolutional Neural Network

### MNIST

In [ ]:
np.random.seed(2020)

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]

In [ ]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [ ]:
trials = 5
tol = -1e-5
layer_sizes = [64, 256, 128, 64, 32]
num_rem_ex = []
num_rem_greed = []
num_rem_greed2 = []
delta_accs_ex = []
delta_losses_ex = []
delta_accs_greed = []
delta_losses_greed = []
delta_accs_greed2 = []
delta_losses_greed2 = []
diff_accs = []
diff_losses = []
diff_rem = []
diff_rem2 = []
diff_accs2 = []
diff_losses2 = []
time_ex = []
time_greed = []
time_greed2 = []

for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    l, a = model.evaluate(x_test, y_test, verbose=0)
    
    ig_cut = None
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut)
    end = time.time()
    num_rem_ex += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_ex += [na - a]
    delta_losses_ex += [nl - l]
    time_ex += [end - start]
    
    ig_cut = -1e-2
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
    end = time.time()
    num_rem_greed += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_greed += [na - a]
    delta_losses_greed += [nl - l]
    time_greed += [end - start]
    
    ig_cut = -1e-3
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
    end = time.time()
    num_rem_greed2 += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_greed2 += [na - a]
    delta_losses_greed2 += [nl - l]
    time_greed2 += [end - start]
    
    diff_accs += [delta_accs_ex[trial] - delta_accs_greed[trial]]
    diff_accs2 += [delta_accs_ex[trial] - delta_accs_greed2[trial]]
    diff_losses += [delta_losses_ex[trial] - delta_losses_greed[trial]]
    diff_losses2 += [delta_losses_ex[trial] - delta_losses_greed2[trial]]
    temp_rem = []
    temp_rem2 = []
    for i in range(len(layer_sizes)):   
        temp_rem += [num_rem_greed[trial][i] - num_rem_ex[trial][i]]
        temp_rem2 += [num_rem_greed2[trial][i] - num_rem_ex[trial][i]]
        
    diff_rem += [temp_rem]
    diff_rem2 += [temp_rem2]

In [ ]:
DR = pd.DataFrame(diff_rem, columns=["Layer 5", "Layer 4", "Layer 3", "Layer 2", "Layer 1"])
DR.to_csv('../../../results/num_removed_CNN_greed_ex_mnist.csv')
DR.describe()

In [ ]:
DR2 = pd.DataFrame(diff_rem2, columns=["Layer 5", "Layer 4", "Layer 3", "Layer 2", "Layer 1"])
DR2.to_csv('../../../results/num_removed_CNN_greed_ex_2_mnist.csv')
DR2.describe()

In [ ]:
A = pd.DataFrame([delta_accs_ex, delta_accs_greed, delta_accs_greed2])
A = A.T
A.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
A.to_csv('../../../results/change_accuracy_CNN_greed_ex_mnist.csv')
A.describe()

In [ ]:
L = pd.DataFrame([delta_losses_ex, delta_losses_greed, delta_losses_greed2])
L = L.T
L.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
L.to_csv('../../../results/change_loss_CNN_greed_ex_mnist.csv')
L.describe()

In [ ]:
DA = pd.Series(diff_accs)
DA.to_csv('../../../results/diff_accuracy_CNN_greed_ex_mnist.csv')
DA.describe()

In [ ]:
DA2 = pd.Series(diff_accs2)
DA2.to_csv('../../../results/diff_accuracy_CNN_greed_ex_2_mnist.csv')
DA2.describe()

In [ ]:
DL = pd.Series(diff_losses)
DL.to_csv('../../../results/diff_loss_CNN_greed_ex_mnist.csv')
DL.describe()

In [ ]:
DL2 = pd.Series(diff_losses2)
DL2.to_csv('../../../results/diff_loss_CNN_greed_ex_2_mnist.csv')
DL2.describe()

In [ ]:
Time = pd.DataFrame([time_ex, time_greed, time_greed2])
Time = Time.T
Time.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
Time.to_csv('../../../results/time_CNN_greed_ex_mnist.csv')
Time.describe()

In [ ]:
diff_rem = [sum(rem) for rem in diff_rem]

In [ ]:
diff_rem2 = [sum(rem) for rem in diff_rem2]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=diff_rem, y=diff_accs, marker='o', color='red', s=60)
plt.title("Accuracy difference in Greedy (-1e-2 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_accuracy_vs_nodes_removed_CNN_greed_ex_mnist.png")

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=diff_rem2, y=diff_accs2, marker='o', color='red', s=60)
plt.title("Accuracy difference in Greedy (-1e-3 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_accuracy_vs_nodes_removed_CNN_greed_ex_2_mnist.png")

In [ ]:
plt.scatter(x=diff_rem, y=diff_losses, marker='o', color='red', s=60)
plt.title("Loss difference in Greedy (-1e-2 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_loss_vs_nodes_removed_CNN_greed_ex_mnist.png")

In [ ]:
plt.scatter(x=diff_rem2, y=diff_losses2, marker='o', color='red', s=60)
plt.title("Loss difference in Greedy (-1e-3 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_loss_vs_nodes_removed_CNN_greed_ex_2_mnist.png")

In [ ]:
tol = -1e-5
ig_cut = -1e-3
layer_sizes = [64, 256, 128, 64, 32]
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
l, a = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
nl, na = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layers-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_CNN_greed_pru_mnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layers-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_CNN_greed_pru_mnist.png")

### Fashion MNIST

In [ ]:
np.random.seed(2020)

In [ ]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]

In [ ]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [ ]:
trials = 5
tol = -1e-5
layer_sizes = [64, 256, 128, 64, 32]
num_rem_ex = []
num_rem_greed = []
num_rem_greed2 = []
delta_accs_ex = []
delta_losses_ex = []
delta_accs_greed = []
delta_losses_greed = []
delta_accs_greed2 = []
delta_losses_greed2 = []
diff_accs = []
diff_losses = []
diff_rem = []
diff_rem2 = []
diff_accs2 = []
diff_losses2 = []
time_ex = []
time_greed = []
time_greed2 = []

for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    l, a = model.evaluate(x_test, y_test, verbose=0)
    
    ig_cut = None
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut)
    end = time.time()
    num_rem_ex += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_ex += [na - a]
    delta_losses_ex += [nl - l]
    time_ex += [end - start]
    
    ig_cut = -1e-2
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
    end = time.time()
    num_rem_greed += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_greed += [na - a]
    delta_losses_greed += [nl - l]
    time_greed += [end - start]
    
    ig_cut = -1e-3
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
    end = time.time()
    num_rem_greed2 += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_greed2 += [na - a]
    delta_losses_greed2 += [nl - l]
    time_greed2 += [end - start]
    
    diff_accs += [delta_accs_ex[trial] - delta_accs_greed[trial]]
    diff_accs2 += [delta_accs_ex[trial] - delta_accs_greed2[trial]]
    diff_losses += [delta_losses_ex[trial] - delta_losses_greed[trial]]
    diff_losses2 += [delta_losses_ex[trial] - delta_losses_greed2[trial]]
    temp_rem = []
    temp_rem2 = []
    for i in range(len(layer_sizes)):   
        temp_rem += [num_rem_greed[trial][i] - num_rem_ex[trial][i]]
        temp_rem2 += [num_rem_greed2[trial][i] - num_rem_ex[trial][i]]
        
    diff_rem += [temp_rem]
    diff_rem2 += [temp_rem2]

In [ ]:
DR = pd.DataFrame(diff_rem, columns=["Layer 5", "Layer 4", "Layer 3", "Layer 2", "Layer 1"])
DR.to_csv('../../../results/num_removed_CNN_greed_ex_fmnist.csv')
DR.describe()

In [ ]:
DR2 = pd.DataFrame(diff_rem2, columns=["Layer 5", "Layer 4", "Layer 3", "Layer 2", "Layer 1"])
DR2.to_csv('../../../results/num_removed_CNN_greed_ex_2_fmnist.csv')
DR2.describe()

In [ ]:
A = pd.DataFrame([delta_accs_ex, delta_accs_greed, delta_accs_greed2])
A = A.T
A.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
A.to_csv('../../../results/change_accuracy_CNN_greed_ex_fmnist.csv')
A.describe()

In [ ]:
L = pd.DataFrame([delta_losses_ex, delta_losses_greed, delta_losses_greed2])
L = L.T
L.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
L.to_csv('../../../results/change_loss_CNN_greed_ex_fmnist.csv')
L.describe()

In [ ]:
DA = pd.Series(diff_accs)
DA.to_csv('../../../results/diff_accuracy_CNN_greed_ex_fmnist.csv')
DA.describe()

In [ ]:
DA2 = pd.Series(diff_accs2)
DA2.to_csv('../../../results/diff_accuracy_CNN_greed_ex_2_fmnist.csv')
DA2.describe()

In [ ]:
DL = pd.Series(diff_losses)
DL.to_csv('../../../results/diff_loss_CNN_greed_ex_fmnist.csv')
DL.describe()

In [ ]:
DL2 = pd.Series(diff_losses2)
DL2.to_csv('../../../results/diff_loss_CNN_greed_ex_2_fmnist.csv')
DL2.describe()

In [ ]:
Time = pd.DataFrame([time_ex, time_greed, time_greed2])
Time = Time.T
Time.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
Time.to_csv('../../../results/time_CNN_greed_ex_fmnist.csv')
Time.describe()

In [ ]:
diff_rem = [sum(rem) for rem in diff_rem]

In [ ]:
diff_rem2 = [sum(rem) for rem in diff_rem2]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=diff_rem, y=diff_accs, marker='o', color='red', s=60)
plt.title("Accuracy difference in Greedy (-1e-2 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_accuracy_vs_nodes_removed_CNN_greed_ex_fmnist.png")

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=diff_rem2, y=diff_accs2, marker='o', color='red', s=60)
plt.title("Accuracy difference in Greedy (-1e-3 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_accuracy_vs_nodes_removed_CNN_greed_ex_2_fmnist.png")

In [ ]:
plt.scatter(x=diff_rem, y=diff_losses, marker='o', color='red', s=60)
plt.title("Loss difference in Greedy (-1e-2 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_loss_vs_nodes_removed_CNN_greed_ex_fmnist.png")

In [ ]:
plt.scatter(x=diff_rem2, y=diff_losses2, marker='o', color='red', s=60)
plt.title("Loss difference in Greedy (-1e-3 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_loss_vs_nodes_removed_CNN_greed_ex_2_fmnist.png")

In [ ]:
tol = -1e-5
ig_cut = -1e-3
layer_sizes = [64, 256, 128, 64, 32]
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
l, a = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
nl, na = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layers-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_CNN_greed_pru_fmnist.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layers-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_CNN_greed_pru_fmnist.png")

### CIFAR10

In [ ]:
np.random.seed(2020)

In [ ]:
cifar = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [ ]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [ ]:
trials = 5
tol = -1e-5
layer_sizes = [64, 256, 128, 64, 32]
num_rem_ex = []
num_rem_greed = []
num_rem_greed2 = []
delta_accs_ex = []
delta_losses_ex = []
delta_accs_greed = []
delta_losses_greed = []
delta_accs_greed2 = []
delta_losses_greed2 = []
diff_accs = []
diff_losses = []
diff_rem = []
diff_rem2 = []
diff_accs2 = []
diff_losses2 = []
time_ex = []
time_greed = []
time_greed2 = []

for trial in tqdm.trange(trials):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    l, a = model.evaluate(x_test, y_test, verbose=0)
    
    ig_cut = None
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut)
    end = time.time()
    num_rem_ex += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_ex += [na - a]
    delta_losses_ex += [nl - l]
    time_ex += [end - start]
    
    ig_cut = -1e-2
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
    end = time.time()
    num_rem_greed += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_greed += [na - a]
    delta_losses_greed += [nl - l]
    time_greed += [end - start]
    
    ig_cut = -1e-3
    
    start = time.time()
    shrinked_weights, _, _, tmp_a, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
    end = time.time()
    num_rem_greed2 += [tmp_a]
    tester_model.set_weights(shrinked_weights)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    delta_accs_greed2 += [na - a]
    delta_losses_greed2 += [nl - l]
    time_greed2 += [end - start]
    
    diff_accs += [delta_accs_ex[trial] - delta_accs_greed[trial]]
    diff_accs2 += [delta_accs_ex[trial] - delta_accs_greed2[trial]]
    diff_losses += [delta_losses_ex[trial] - delta_losses_greed[trial]]
    diff_losses2 += [delta_losses_ex[trial] - delta_losses_greed2[trial]]
    temp_rem = []
    temp_rem2 = []
    for i in range(len(layer_sizes)):   
        temp_rem += [num_rem_greed[trial][i] - num_rem_ex[trial][i]]
        temp_rem2 += [num_rem_greed2[trial][i] - num_rem_ex[trial][i]]
        
    diff_rem += [temp_rem]
    diff_rem2 += [temp_rem2]

In [ ]:
DR = pd.DataFrame(diff_rem, columns=["Layer 5", "Layer 4", "Layer 3", "Layer 2", "Layer 1"])
DR.to_csv('../../../results/num_removed_CNN_greed_ex_cifar.csv')
DR.describe()

In [ ]:
DR2 = pd.DataFrame(diff_rem2, columns=["Layer 5", "Layer 4", "Layer 3", "Layer 2", "Layer 1"])
DR2.to_csv('../../../results/num_removed_CNN_greed_ex_2_cifar.csv')
DR2.describe()

In [ ]:
A = pd.DataFrame([delta_accs_ex, delta_accs_greed, delta_accs_greed2])
A = A.T
A.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
A.to_csv('../../../results/change_accuracy_CNN_greed_ex_cifar.csv')
A.describe()

In [ ]:
L = pd.DataFrame([delta_losses_ex, delta_losses_greed, delta_losses_greed2])
L = L.T
L.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
L.to_csv('../../../results/change_loss_CNN_greed_ex_cifar.csv')
L.describe()

In [ ]:
DA = pd.Series(diff_accs)
DA.to_csv('../../../results/diff_accuracy_CNN_greed_ex_cifar.csv')
DA.describe()

In [ ]:
DA2 = pd.Series(diff_accs2)
DA2.to_csv('../../../results/diff_accuracy_CNN_greed_ex_2_cifar.csv')
DA2.describe()

In [ ]:
DL = pd.Series(diff_losses)
DL.to_csv('../../../results/diff_loss_CNN_greed_ex_cifar.csv')
DL.describe()

In [ ]:
DL2 = pd.Series(diff_losses2)
DL2.to_csv('../../../results/diff_loss_CNN_greed_ex_2_cifar.csv')
DL2.describe()

In [ ]:
Time = pd.DataFrame([time_ex, time_greed, time_greed2])
Time = Time.T
Time.columns = ['Exhaustive', 'Greedy (-1e-2 cutoff)', 'Greedy (-1e-3 cutoff)']
Time.to_csv('../../../results/time_CNN_greed_ex_cifar.csv')
Time.describe()

In [ ]:
diff_rem = [sum(rem) for rem in diff_rem]

In [ ]:
diff_rem2 = [sum(rem) for rem in diff_rem2]

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=diff_rem, y=diff_accs, marker='o', color='red', s=60)
plt.title("Accuracy difference in Greedy (-1e-2 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_accuracy_vs_nodes_removed_CNN_greed_ex_cifar.png")

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.scatter(x=diff_rem2, y=diff_accs2, marker='o', color='red', s=60)
plt.title("Accuracy difference in Greedy (-1e-3 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_accuracy_vs_nodes_removed_CNN_greed_ex_2_cifar.png")

In [ ]:
plt.scatter(x=diff_rem, y=diff_losses, marker='o', color='red', s=60)
plt.title("Loss difference in Greedy (-1e-2 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_loss_vs_nodes_removed_CNN_greed_ex_cifar.png")

In [ ]:
plt.scatter(x=diff_rem2, y=diff_losses2, marker='o', color='red', s=60)
plt.title("Loss difference in Greedy (-1e-3 cutoff) versus Exhaustive pruning", size=28)
plt.xlabel("Difference in nodes removed", size=20)
plt.xticks(fontsize=20)
plt.ylabel("Difference in Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
plt.savefig("../../../figures/diff_loss_vs_nodes_removed_CNN_greed_ex_2_cifar.png")

In [ ]:
tol = -1e-5
ig_cut = -1e-3
layer_sizes = [64, 256, 128, 64, 32]
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5, verbose=0)
shrinked_weights, acc_ev, loss_ev, num_rem, _ = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
l, a = model.evaluate(x_test, y_test, verbose=0)
tester_model.set_weights(shrinked_weights)
nl, na = tester_model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print(f"Original loss: {l}, New loss: {nl}")
print(f"Original accuracy: {a}, New accuracy: {na}")
print(f"Number of nodes removed: {num_rem}")

In [ ]:
changes = []
tot_rem = 0
for rem in num_rem:
    tot_rem += rem
    changes.append(tot_rem)
colors = ["black", "blue", "red", "green"]

In [ ]:
plt.plot(acc_ev)
plt.title("Evolution of accuracy as nodes are removed", size=28)
plt.ylabel("Accuracy", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layers-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_acc_CNN_greed_pru_cifar.png")

In [ ]:
plt.plot(loss_ev)
plt.title("Evolution of loss as nodes are removed", size=28)
plt.ylabel("Loss", size=20)
plt.yticks(fontsize=20)
plt.grid(True)
num_layers = len(changes)
for i, pos in enumerate(changes[:-1]):
    plt.axvline(x=pos, label=f"Layer {num_layers-i} -> Layer {num_layers-(i+1)}", color=colors[i])
plt.legend(fontsize=20)
plt.savefig("../../../figures/ev_loss_CNN_greed_pru_cifar.png")